<a href="https://colab.research.google.com/github/mahfouz21/RxClose-Chatbot/blob/main/Chatbot_v_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install -U langchain-community
!pip install ctransformers
!pip install pinecone-client
!pip install sentence-transformers
!pip install transformers
!pip install python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 455, in run
    installed = install_given_reqs(
                ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/__init__.py", line 70, in install_given_reqs
    requirement.install(
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/req/req_install.py", line 851, in install
    install_wheel(
  File "/usr/local/lib/pyt

In [ ]:


# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import os
import pinecone
import pandas as pd
import json
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.vectorstores import Pinecone as LangchainPinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set environment variables
os.environ["PINECONE_API_KEY"] = "pcsk_29YpbD_SHWVCkHpPQMnuto42uEqyduYCM2D3SeUNV5guDWeMNrj71rZMASqzNr26EvisMJ"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/GP/Medical_Intelligence_Dataset.csv")

# Clean and split data
df = df.dropna(subset=['input'])
df = df[df['input'].str.strip() != '']

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=20
)

df['chunks'] = df['input'].apply(lambda x: text_splitter.split_text(x) if isinstance(x, str) and pd.notna(x) and len(x) > 0 else [])
df_chunked = df.explode('chunks', ignore_index=True)

# Download embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Initialize Pinecone
pc = pinecone.Pinecone(
    api_key=os.environ["PINECONE_API_KEY"],
    environment=os.environ["PINECONE_ENVIRONMENT"]
)

# Define index name
index_name = "medicalchatbot"

# Store embeddings in Pinecone
docsearch = LangchainPinecone.from_texts(
    df_chunked['chunks'].tolist(),
    embeddings,
    index_name=index_name
)

# Save configurations
config = {
    "chunk_size": 500,
    "chunk_overlap": 20,
    "prompt_template": '''Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
'''
}

# Save config to Google Drive
with open("/content/drive/MyDrive/GP/chatbot_config.json", "w") as f:
    json.dump(config, f)

print("Initial setup complete! Pinecone index and configurations saved.")

KeyboardInterrupt: 

#Reload the chatbot and start

In [ ]:
# Install required libraries (if not already installed)
!pip install -U langchain-community
!pip install ctransformers
!pip install pinecone-client
!pip install sentence-transformers
!pip install transformers

  Using cached langchain_community-0.3.16-py3-none-any.whl.metadata (2.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
  Using cached pydantic_settings-2.7.1-py3-none-any.whl.metadata (3.5 kB)
  Using cached marshmallow-3.26.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
Using cached langchain_community-0.3.16-py3-none-any.whl (2.5 MB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx_sse-0.4.0-py3-none-any.whl (7.8 kB)
Using cached pydantic_settings-2.7.1-py3-none-any.whl (29 kB)
Using cached marshmallow-3.26.0-py3-none-any.whl (50 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pinecone
import json
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import CTransformers
from langchain.vectorstores import Pinecone as LangchainPinecone

# Load configurations
with open("/content/drive/MyDrive/GP/chatbot_config.json", "r") as f:
    config = json.load(f)

chunk_size = config["chunk_size"]
chunk_overlap = config["chunk_overlap"]
prompt_template = config["prompt_template"]

# Set environment variables
os.environ["PINECONE_API_KEY"] = "pcsk_29YpbD_SHWVCkHpPQMnuto42uEqyduYCM2D3SeUNV5guDWeMNrj71rZMASqzNr26EvisMJ"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"

# Initialize Pinecone
pc = pinecone.Pinecone(
    api_key=os.environ["PINECONE_API_KEY"],
    environment=os.environ["PINECONE_ENVIRONMENT"]
)

# Load existing index
index_name = "medicalchatbot"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
docsearch = LangchainPinecone.from_existing_index(index_name, embeddings)

# Load LLM
llm = CTransformers(
    model="TheBloke/Llama-2-7B-Chat-GGML",
    model_type="llama",
    config={
        'max_new_tokens': 512,
        'temperature': 0.8
    }
)

# Recreate RetrievalQA chain
PROMPT = PromptTemplate(template=prompt_template, input_variables=['context', 'question'])
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs,
)

<ipython-input-6-0f30f5cb8c84>:31: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

In [ ]:
# Run user interface
print("Chatbot is ready! Type 'exit' to stop.")
while True:
    user_input = input("input prompt: ")
    if user_input.lower() == "exit":
        break
    result = qa({"query": user_input})
    print("response:", result['result'])

Chatbot is ready! Type 'exit' to stop.
input prompt: hi


<ipython-input-2-405d14dcb6f3>:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": user_input})


response: The time it takes for a letter to reach another person depends on various factors such as the distance between the two people, the type of mail service used, and the delivery speed chosen by the sender. For example, if you send a letter through regular mail, it might take a few days to a week to arrive, while sending a letter through express mail or overnight delivery may take just a few hours to a day or two at most.
input prompt: i have a sore thorat
response: - Do you have any other symptoms such as fever, fatigue, or weight loss?
- Have you recently traveled or been exposed to anyone with cold/flu like symptoms?
input prompt: exit


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr


def chatbot_response(user_input):
    if user_input.lower() == "exit":
        return "Chatbot stopped. Refresh the page to restart."

    result = qa({"query": user_input})
    return result['result']


iface = gr.Interface(
    fn=chatbot_response,
    inputs=gr.Textbox(lines=2, placeholder="Enter your question..."),
    outputs="text",
    title="Pharmacy Chatbot",
    description="Ask me about symptoms, diseases, or medications!"
)


iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2419edf873e2a9197.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# # Install required libraries
# !pip install -U langchain-community ctransformers pinecone-client sentence-transformers transformers python-dotenv asyncio

# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Import libraries
# import os
# import pinecone
# import pandas as pd
# import json
# import asyncio
# from functools import lru_cache
# from langchain import PromptTemplate
# from langchain.chains import RetrievalQA
# from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.llms import CTransformers
# from langchain.vectorstores import Pinecone as LangchainPinecone
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from dotenv import load_dotenv

# # Set environment variables
# os.environ["PINECONE_API_KEY"] = "pcsk_29YpbD_SHWVCkHpPQMnuto42uEqyduYCM2D3SeUNV5guDWeMNrj71rZMASqzNr26EvisMJ"  # Replace with your Pinecone API key
# os.environ["PINECONE_ENVIRONMENT"] = "us-east-1"  # Replace with your Pinecone environment

# # Initialize Pinecone
# pc = pinecone.Pinecone(
#     api_key=os.environ["PINECONE_API_KEY"],
#     environment=os.environ["PINECONE_ENVIRONMENT"]
# )

# # Load dataset
# try:
#     df = pd.read_csv("/content/drive/MyDrive/GP/Medical_Intelligence_Dataset.csv")
# except Exception as e:
#     raise Exception(f"Error loading dataset: {e}")

# # Clean and split data
# df = df.dropna(subset=['input'])
# df = df[df['input'].str.strip() != '']

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,
#     chunk_overlap=20
# )

# # Split text into chunks
# df['chunks'] = df['input'].apply(lambda x: text_splitter.split_text(x) if isinstance(x, str) and pd.notna(x) and len(x) > 0 else [])
# df_chunked = df.explode('chunks', ignore_index=True)

# # Download embeddings
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# # # Initialize Pinecone
# # try:
# #     pc = pinecone.Pinecone(
# #         api_key=pinecone_api_key,
# #         environment=pinecone_environment
# #     )
# # except Exception as e:
# #     raise Exception(f"Error initializing Pinecone: {e}")

# # Define index name
# index_name = "medicalchatbot"

# # # Check if index exists, otherwise create it
# # if index_name not in pc.list_indexes():
# #     try:
# #         # Create index with PodSpec
# #         pc.create_index(
# #             name=index_name,
# #             dimension=384,  # Dimension of the embeddings
# #             spec=pinecone.PodSpec(
# #                 environment=pinecone_environment,  # Pass the environment here
# #                 pod_type="p1"  # Specify the pod type
# #             )
# #         )
# #     except Exception as e:
# #         raise Exception(f"Error creating Pinecone index: {e}")

# # Store embeddings in Pinecone in batches
# batch_size = 1000
# for i in range(0, len(df_chunked), batch_size):
#     batch = df_chunked[i:i + batch_size]
#     try:
#         LangchainPinecone.from_texts(
#             batch['chunks'].tolist(),
#             embeddings,
#             index_name=index_name
#         )
#     except Exception as e:
#         print(f"Error storing batch {i} in Pinecone: {e}")

# # Save configurations
# config = {
#     "chunk_size": 500,
#     "chunk_overlap": 20,
#     "prompt_template": """
# You are a medical chatbot designed to provide accurate and helpful information. Use the following context to answer the user's question. If you don't know the answer, say "I don't know."

# Context: {context}
# Question: {question}

# Answer:
# """
# }

# # Save config to Google Drive
# try:
#     with open("/content/drive/MyDrive/GP/chatbot_config2.json", "w") as f:
#         json.dump(config, f)
# except Exception as e:
#     print(f"Error saving config file: {e}")

# print("Initial setup complete! Pinecone index and configurations saved.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Initial setup complete! Pinecone index and configurations saved.


In [ ]:
# # Load existing index and embeddings
# try:
#     docsearch = LangchainPinecone.from_existing_index(index_name, embeddings)
# except Exception as e:
#     raise Exception(f"Error loading Pinecone index: {e}")

# # Load LLM
# try:
#     llm = CTransformers(
#         model="TheBloke/Llama-2-7B-Chat-GGML",
#         model_type="llama",
#         config={
#             'max_new_tokens': 512,
#             'temperature': 0.8
#         }
#     )
# except Exception as e:
#     raise Exception(f"Error loading LLM: {e}")

# # Recreate RetrievalQA chain
# PROMPT = PromptTemplate(template=config["prompt_template"], input_variables=['context', 'question'])
# chain_type_kwargs = {"prompt": PROMPT}

# try:
#     qa = RetrievalQA.from_chain_type(
#         llm=llm,
#         chain_type="stuff",
#         retriever=docsearch.as_retriever(search_kwargs={"k": 2}),
#         return_source_documents=True,
#         chain_type_kwargs=chain_type_kwargs,
#     )
# except Exception as e:
#     raise Exception(f"Error creating RetrievalQA chain: {e}")

# # Save the QA chain for future use
# try:
#     import pickle
#     with open("/content/drive/MyDrive/GP/qa_chain.pkl", "wb") as f:
#         pickle.dump(qa, f)
# except Exception as e:
#     print(f"Error saving QA chain: {e}")

# # Add caching for frequent queries
# @lru_cache(maxsize=100)
# def cached_qa_query(query: str):
#     return qa({"query": query})

# # Asynchronous query processing
# async def process_query_async(query: str):
#     try:
#         result = cached_qa_query(query)
#         return result
#     except Exception as e:
#         print(f"Error processing query: {e}")
#         return None

# # Batch processing for multiple queries
# async def process_queries_batch(queries: list):
#     tasks = [process_query_async(query) for query in queries]
#     results = await asyncio.gather(*tasks)
#     return results

# # Run user interface
# print("Chatbot is ready! Type 'exit' to stop.")
# while True:
#     user_input = input("input prompt: ")
#     if user_input.lower() == "exit":
#         break
#     try:
#         result = asyncio.run(process_query_async(user_input))
#         print("response:", result['result'])
#         print("Sources:")
#         for doc in result['source_documents']:
#             print(f"- {doc.page_content[:100]}...")  # Display a snippet of the source document
#     except Exception as e:
#         print(f"Error generating response: {e}")

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.ggmlv3.q2_K.bin:   0%|          | 0.00/2.87G [00:00<?, ?B/s]

Error saving QA chain: Can't pickle local object 'CDLL.__init__.<locals>._FuncPtr'
Chatbot is ready! Type 'exit' to stop.
input prompt: i have a headache


<ipython-input-8-9d36757ec572>:76: RuntimeWarning: coroutine 'process_query_async' was never awaited
  print(f"Error generating response: {e}")


Error generating response: asyncio.run() cannot be called from a running event loop
input prompt: hi
Error generating response: asyncio.run() cannot be called from a running event loop
input prompt: i have a headache
Error generating response: asyncio.run() cannot be called from a running event loop
input prompt: exit
